#### Initial Setup

In [1]:
#imported libraries
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import boto
import numpy as np
from numpy import matlib
import scipy as sp
import math
import matplotlib.pyplot as plt
import langdetect
import datetime
import warnings
%matplotlib inline  
from sklearn.svm import SVR
from sklearn import linear_model
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import Imputer

import os
import findspark; findspark.init()
import pyspark
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql.types import Row
import pyspark.sql.functions as func
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from pyspark.mllib.util import MLUtils

from pyspark.mllib.linalg import Vectors 
from pyspark.mllib.regression import LabeledPoint  
from pyspark.mllib.regression import LinearRegressionWithSGD 

os.environ["PYSPARK_SUBMIT_ARGS"] = (
  "--packages com.databricks:spark-csv_2.11:1.4.0 pyspark-shell"
)

#NEED TO ADD "  SPARK_DRIVER_MEMORY=5G   "  to ./conf/spark-env.sh 

try:
    conf = SparkConf().set("spark.executor.memory", "3g")
    sc = pyspark.SparkContext(conf=conf)
except Exception as e:
    print "SparkContext exists... Continuing on."
    
sqlCtx = pyspark.sql.SQLContext(sc)
sc.setCheckpointDir('checkpoint/')

In [2]:
try:
    output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/train_final_downloads.csv').drop('')
    gdp_countries_pd = pd.read_csv('s3://cs341bucket1/Data/GDP.csv')
except:
    output = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('train_final_downloads.csv').drop('')
    gdp_countries_pd = pd.read_csv('GDP.csv')
output = output.withColumnRenamed("cumulative_downloads_2016-02","cumulative_downloads_2016_02")


In [3]:
def testing(predictors):
    # use this to print information about predictor at each step
    return None

In [4]:

def get_log_week(*args):
    args = [x if not (x<0) else 0 for x in list(args)]
    nb_0 = args.count(0)
    if nb_0 == 7:
        return float(0)
    return math.log(1.0*sum(args)/(7-nb_0))
    #Y = [np.log(c) for c in args if c > 0]
    #if len(Y) == 0 : return 0
    #return float(1.0*sum(Y))/(7-nb_0)
sqlCtx.registerFunction("get_log_week", get_log_week,returnType=FloatType())
def get_download_sum(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    return (1.0*sum(args))
sqlCtx.registerFunction("get_download_sum", get_download_sum,returnType=FloatType())


def get_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = [x if not (x==-1) else 0 for x in list(args)]    
    return  float(np.polyfit(range(56),np.cumsum(args[1:]),3)[3-args[0]])
    #Y = [np.log(c) for c in args[1:] if c>0]
    #if len(Y)<=1: return float(0)
    #return  float(np.polyfit(range(len(Y)),np.cumsum(Y),args[0])[0])

#Generate the step max and min 
def get_maxStep(maximum,*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    if (np.count_nonzero(args) == 0):
        return float(0)
    if maximum :
        return max(args)
    else:
        return min(filter(lambda x: x>0,args))


def get_maxStep_old(maximum,*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    m = 0
    for d in range(1,56):
        if (args[d]!=0 and args[d-1]!=0):
            c = (args[d]-args[d-1])
            #c = float(args[d])/args[d-1]
            if (maximum and m < c):
                m = c
            if ( not maximum and m > c):
                m = c
    return m
    #if m==0: return float(0)
    #return m

def get_std(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    return float(np.std(list(args)))

def get_nbMissing(*args):
    return list(args).count(-1)

#Generate the daily average

def get_revenue_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    time_series = [np.log(c) for c in args[1:] if c>0]
    if len(time_series) <=1: return float(0)
    #return float(np.polyfit(np.array(range(56)),args[1:],args[0])[0])
    return  float(np.polyfit(np.array(range(len(time_series))),time_series,args[0])[0])

def get_usage_coefficients(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    if -1 in args: return 0
    return  float(np.polyfit(range(8),args[1:],args[0])[0])

def get_usage_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.max(args))

def get_usage_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    args = [c for c in args[1:] if c!=-1]
    if len(args)==0: return 0
    return  float(np.mean(args))

def get_revenue_max(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    time_series = [np.log(c) for c in args[1:] if c>0]
    if len(time_series) <=1: return float(0)
    #args = [c for c in args[1:] if c!=-1]
    #if len(args)==0: return 0
    #return  float(np.max(args))
    return float(np.max(time_series))

def get_revenue_mean(*args):
    #The first element of the list is the degree of the coefficient
    args = list(args)
    #args = [c for c in args[1:] if c!=-1]
    #if len(args)==0: return 0
    #return  float(np.mean(args))
    time_series = [np.log(c) for c in args[1:] if c>0]
    if len(time_series) <=1: return float(0)
    return float(np.mean(time_series))

def get_dailyAvg(*args):
    args = [x if not (x==-1) else 0 for x in list(args)]
    if (np.count_nonzero(args) == 0):
        return float(0)
    return  (1.0*sum(args)/np.count_nonzero(args))
    #return float(sum([np.log(c) for c in args if c>0]))/np.count_nonzero(args)

sqlCtx.registerFunction("get_nbMissing", get_nbMissing,returnType=IntegerType())
sqlCtx.registerFunction("get_std", get_std,returnType=FloatType())
sqlCtx.registerFunction("get_maxStep", get_maxStep,returnType=IntegerType())
sqlCtx.registerFunction("get_maxStep_old", get_maxStep_old,returnType=FloatType())
sqlCtx.registerFunction("get_coefficients", get_coefficients,returnType=FloatType())
sqlCtx.registerFunction("daily_avg", get_dailyAvg,returnType=FloatType())
sqlCtx.registerFunction("get_usage_coefficients", get_usage_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_coefficients", get_revenue_coefficients,returnType=FloatType())
sqlCtx.registerFunction("get_usage_max", get_usage_max,returnType=FloatType())
sqlCtx.registerFunction("get_usage_mean", get_usage_mean,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_max", get_revenue_max,returnType=FloatType())
sqlCtx.registerFunction("get_revenue_mean", get_revenue_mean,returnType=FloatType())

lang = ['ja','zh-cn','ko','en','other']
def get_language(x):
    try:
        detected = langdetect.detect_langs(unicode(x))[0]
        if detected.prob < 0.7:
            return "other"
        elif  detected.lang in lang:
            return detected.lang
        else:
            return "other"
    except:
        return "other"
sqlCtx.registerFunction("get_language", get_language,returnType=StringType())

def get_days(date, id):
    if (date == "no_date") or True:   #Attention here, only imputation
        # return 0
        id = 1.0*id/100000000
        return int(5856.25394104 -1731.74798728*id+195.553086*id**2  -8.12861635*id**3)
    else:
        try:
            return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
                - datetime.datetime.strptime(date, '%m/%d/%Y').date()).days
        except:
            return (datetime.datetime.strptime('03/01/2015', '%m/%d/%Y').date() \
                - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days
sqlCtx.registerFunction("get_days", get_days,returnType=IntegerType())

#escape is used in case some asshole used - or [space] anywhere
def escape(text):
    return text.replace(" ","_").replace("-","_")
# number of reviews
def get_recentReviews(date):
    return int((datetime.datetime.strptime('04/01/2015', '%m/%d/%Y').date() \
            - datetime.datetime.strptime(date, '%Y-%m-%d').date()).days >=0)

def get_market(country,device):
    #http://blog.nelso.com/2010/06/iphone-os-penetration-by-country.html
    if device == "iphone":
        return {
            "United_States" : 10683403,
            "France" : 2248817,
            "Japan" : 1378903,
            "Spain" : 377346,
            "United_Kingdom" : 2551128,
            "Germany" : 1117716,
            "Hong_Kong" : 299720,
            "Switzerland" : 399364,
            "Netherlands" : 372539,
            "Australia" : 1207428,
            "Norway" : 154218,
            "Sweden" : 281622,
            "China" : 725358,
            "Canada" : 919074,
            "Denmark" : 151426,
            "Italy" : 648718,
            "Taiwan" : 174226,
            "Mexico" : 215326,
            "Austria" : 156322,
            "Brazil" : 219339,
            "Poland" : 72114,
            "Singapore" : 402922,
            "Hungary" : 33219,
            "Czech_Republic" : 42753,
            'South_Korea': 530235,
            "Russia" :246421   
    }.get(country, 15000) 
    else:
        return{
            "United_States" : 223269,
            "France" : 2724,
            "Japan" : 2293,
            "Spain" : 1494,
            "United_Kingdom" : 4197,
            "Germany" : 3403,
            "Hong_Kong" : 2306,
            "Switzerland" : 1698,
            "Netherlands" : 2554,
            "Australia" : 1400,
            "Norway" : 1333,
            "Sweden" : 1188,
            "China" : 12516,
            "Canada" : 6275,
            "Denmark" : 753,
            "Italy" : 1370,
            "Taiwan" : 1356,
            "Mexico" : 3380,
            "Austria" : 493,
            "Brazil" : 2014,
            "Poland" : 324,
            "Singapore" : 1453,
            "Hungary" : 211,
            "Czech_Republic" : 203,
            'South_Korea': 2416,
            "Russia" :2183
        }.get(country, 100) 
    return 1
sqlCtx.registerFunction("get_market", get_market,returnType=IntegerType())

def get_gdp(country):
    if country == "no_country":
        return 0.0
    try:
        return float(gdp_countries_pd[gdp_countries_pd.Country == country]["GDP2015"])
    
    except:
        return 0.0
sqlCtx.registerFunction("get_gdp", get_gdp,returnType=FloatType())

In [ ]:
def generate_predictors(down, rati, usag, reve, prev, rele, raco, revi ):



    ###load files : 
    try:
        sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID'])
        sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])
        downloads = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+down).drop('')
        ratings = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+rati).drop('')
        usages = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+usag).drop('')
        revenues = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+reve).drop('')
        prev_downloads = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+prev).drop('')  
        release_date = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+rele).drop('')
        rating_country = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load('s3n://cs341bucket1/Data/'+raco).drop('')

        reviews_schema = StructType([
            StructField("id",IntegerType(),True),
            StructField("name",StringType(),True),
            StructField("country",StringType(),True),
            StructField("rating",IntegerType(),True),
            StructField("date",StringType(),True),
            StructField("title",StringType(),True),
            StructField("version",StringType(),True),
            StructField("text",StringType(),True),
            StructField("reviewer",StringType(),True)
        ])
        reviews = pd.read_csv('s3://cs341bucket1/Data/'+revi)
        reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

    except:
        downloads = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(down).drop('')
        ratings = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(rati).drop('')
        usages = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(usag).drop('')
        revenues = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(reve).drop('')
        prev_downloads = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(prev).drop('')  
        release_date = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(rele).drop('')

        rating_country = sqlCtx.read \
            .format('com.databricks.spark.csv') \
            .options(header='true',inferSchema='true') \
            .load(raco).drop('')

        reviews_schema = StructType([
            StructField("id",IntegerType(),True),
            StructField("name",StringType(),True),
            StructField("country",StringType(),True),
            StructField("rating",IntegerType(),True),
            StructField("date",StringType(),True),
            StructField("title",StringType(),True),
            StructField("version",StringType(),True),
            StructField("text",StringType(),True),
            StructField("reviewer",StringType(),True)
        ])
        reviews = pd.read_csv(revi)
        reviews = sqlCtx.createDataFrame(reviews,reviews_schema)

    
    ###imputations
    #usage imputation
    imputer = Imputer(missing_values=-1, strategy='median', axis=0)
    pd_usages = usages.toPandas()
    category = list(set(pd_usages["category"].values))
    imp = pd.DataFrame(columns = pd_usages.columns)
    for cat in category:
        #for dev in ["iphone","ipad"]:
            for metric in range(1,5):
                curr_df = pd_usages.ix[pd_usages["category"]==cat,:]
                #curr_df = curr_df.ix[curr_df["device"]==dev,:]
                curr_df = curr_df.ix[curr_df["metric"]==metric,:]
                try:
                    name = curr_df.columns
                    df1 = curr_df.ix[:,0:6]
                    df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
                    df2.index = df1.index
                    curr_df = pd.concat([df1,df2],axis = 1)
                    curr_df.columns = name 
                except:
                    pass
                imp = pd.concat([imp,curr_df],axis = 0)
    usages = sqlCtx.createDataFrame(imp).fillna(-1)

    #revenue imputation
    pd_revenues = revenues.toPandas()
    imp = pd.DataFrame(columns = revenues.columns)
    for cat in category:
        for dev in ["iphone","ipad"]:
            #for metric in range(1,5):
                curr_df = pd_revenues.ix[pd_revenues["category"]==cat,:]
                curr_df = curr_df.ix[curr_df["device"]==dev,:]
                #curr_df = curr_df.ix[curr_df["metric"]==metric,:]
                name = curr_df.columns
                df1 = curr_df.ix[:,0:5]
                df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
                df2.index = df1.index
                curr_df = pd.concat([df1,df2],axis = 1)
                curr_df.columns = name 
                imp = pd.concat([imp,curr_df],axis = 0)
    revenues = sqlCtx.createDataFrame(imp).fillna(-1)



    #warnings.filterwarnings('error')

    ### Renaming
    old_dateRange = pd.date_range('03/01/2015', periods=56).format(formatter=lambda x: x.strftime('%Y-%m-%d'))
    dateRange = pd.date_range('03/01/2015', periods=56).format(formatter=lambda x: x.strftime('%m_%d_%Y'))
    for d in range(56):
        revenues = revenues.withColumnRenamed(old_dateRange[d],dateRange[d])
        usages = usages.withColumnRenamed(old_dateRange[d],dateRange[d])
        downloads = downloads.withColumnRenamed(old_dateRange[d],dateRange[d])  
    prev_downloads = prev_downloads.withColumnRenamed("cumulative_downloads_2015-02","cumulative_downloads_2015_02")
    try:
        ratings = ratings.withColumnRenamed("start1","star1")
    except:
        pass

    #Initialization
    predictors = downloads['id','name','category','device']
    testing(predictors)

    # Generate the weekly downloads

    sqlCtx.registerDataFrameAsTable(downloads, "downloads")
    sqlCtx.registerDataFrameAsTable(usages, "usages")
    sqlCtx.registerDataFrameAsTable(revenues, "revenues")



    predictors = sqlCtx.sql("SELECT id, name, category, device, \
               get_log_week("+ ",".join(dateRange[0:7])+") AS week_1 \
                ,get_log_week("+",".join(dateRange[7:14])+") AS week_2 \
                ,get_log_week("+ ",".join(dateRange[14:21])+") AS week_3 \
               ,get_log_week("+",".join(dateRange[21:28])+") AS week_4 \
               ,get_log_week("+",".join(dateRange[28:35])+") AS week_5 \
               ,get_log_week("+",".join(dateRange[35:42])+") AS week_6 \
               ,get_log_week("+",".join(dateRange[42:49])+") AS week_7 \
               ,get_log_week("+",".join(dateRange[49:56])+") AS week_8\
               ,get_download_sum("+ ",".join(dateRange)+") AS download_sum \
               from downloads")
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")

    ###Usages
    m1 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 1")
    m2 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 2")
    m3 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 3")
    m4 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 4")
    sqlCtx.registerDataFrameAsTable(m1,"m1")
    sqlCtx.registerDataFrameAsTable(m2,"m2")
    sqlCtx.registerDataFrameAsTable(m3,"m3")
    sqlCtx.registerDataFrameAsTable(m4,"m4")
    #sqlCtx.registerDataFrameAsTable(avg_score,"avg_score")

    ### Make coefficients

    temp_downloads = sqlCtx.sql("SELECT id,name,category, device \
    , get_coefficients(0,"+",".join(dateRange)+") AS coef_0 \
    ,get_coefficients(1,"+",".join(dateRange)+") AS coef_1 \
    ,get_coefficients(2,"+",".join(dateRange)+") AS coef_2 \
    ,get_coefficients(3,"+",".join(dateRange)+") AS coef_3 \
    ,get_maxStep(True,"+",".join(dateRange)+") AS max_step \
    ,get_maxStep(False,"+",".join(dateRange)+") AS min_step \
    ,get_maxStep_old(True,"+",".join(dateRange)+") AS max_step_old \
    ,get_maxStep_old(False,"+",".join(dateRange)+") AS min_step_old \
    ,get_std("+",".join(dateRange)+") AS downloads_std \
    ,get_nbMissing("+",".join(dateRange)+") AS nb_missing \
    ,daily_avg(" + ",".join(dateRange[0:56]) + ") AS daily_avg \
     FROM downloads")

    predictors = predictors.join(temp_downloads,["id","name","category","device"],how='left_outer')
    testing(predictors)

    m_col = '03_01_2015,03_08_2015,03_15_2015,03_22_2015,03_29_2015,04_05_2015,04_12_2015,04_19_2015'
    
    temp_m1 = sqlCtx.sql("SELECT id, name, category, device, \
    get_usage_coefficients(0,"+m_col+") AS m1_coef_0, \
    get_usage_coefficients(1,"+m_col+") AS m1_coef_1, \
    get_usage_coefficients(2,"+m_col+") AS m1_coef_2, \
    get_usage_max(0,"+m_col+") AS m1_max, \
    get_usage_mean(0,"+m_col+") AS m1_mean FROM m1")

    
    temp_m2 = sqlCtx.sql("SELECT id, name, category, device, \
    get_usage_coefficients(0,"+m_col+") AS m2_coef_0, \
    get_usage_coefficients(1,"+m_col+") AS m2_coef_1, \
    get_usage_coefficients(2,"+m_col+") AS m2_coef_2, \
    get_usage_max(0,"+m_col+") AS m2_max, \
    get_usage_mean(0,"+m_col+") AS m2_mean FROM m2")

    
    temp_m3 = sqlCtx.sql("SELECT id, name, category, device, \
    get_usage_coefficients(0,"+m_col+") AS m3_coef_0, \
    get_usage_coefficients(1,"+m_col+") AS m3_coef_1, \
    get_usage_coefficients(2,"+m_col+") AS m3_coef_2, \
    get_usage_max(0,"+m_col+") AS m3_max, \
    get_usage_mean(0,"+m_col+") AS m3_mean FROM m3")

    
    temp_m4 = sqlCtx.sql("SELECT id, name, category, device, \
    get_usage_coefficients(0,"+m_col+") AS m4_coef_0, \
    get_usage_coefficients(1,"+m_col+") AS m4_coef_1, \
    get_usage_coefficients(2,"+m_col+") AS m4_coef_2, \
    get_usage_max(0,"+m_col+") AS m4_max, \
    get_usage_mean(0,"+m_col+") AS m4_mean FROM m4")


    temp_revenues = sqlCtx.sql("SELECT id, name, category, device, \
    get_revenue_coefficients(0,"+",".join(revenues.columns[4:])+") AS rev_coef_0, \
    get_revenue_coefficients(1,"+",".join(revenues.columns[4:])+") AS rev_coef_1, \
    get_revenue_coefficients(2,"+",".join(revenues.columns[4:])+") AS rev_coef_2, \
    get_revenue_max(0,"+",".join(revenues.columns[4:])+") AS rev_max, \
    get_revenue_mean(0,"+",".join(revenues.columns[4:])+") AS rev_mean FROM revenues")



    predictors = predictors.join(temp_revenues,["id", "name", "category","device"],how='left_outer')
    testing(predictors)
    predictors = predictors.join(temp_m1,["id", "name", "category","device"],how='left_outer')
    testing(predictors)
    predictors = predictors.join(temp_m2,["id", "name", "category","device"],how='left_outer')
    testing(predictors)
    predictors = predictors.join(temp_m3,["id", "name", "category","device"],how='left_outer')
    testing(predictors)
    predictors = predictors.join(temp_m4,["id", "name", "category","device"],how='left_outer')
    #predictors = predictors.join(avg_score,["id", "name", "category","device"],how='left_outer')
    #predictors = predictors.join(temp_usages,["id", "name", "category","device"],how='left_outer')
    #predictors = predictors.join(,)

    testing(predictors)
    
    # previous downloads addition
    predictors = predictors.join(prev_downloads,["id","device"],how='left_outer')

    # Days since release generation with imputation

    testing(predictors)

    release_date = downloads[["id"]].dropDuplicates().join(release_date,["id"],"left").fillna("no_date",["release_date"])
    sqlCtx.registerDataFrameAsTable(release_date, "release_date")

    temp_date = sqlCtx.sql("SELECT id\
    , get_days(release_date, id) AS days_since_release \
     FROM release_date")

    predictors = predictors.join(temp_date,["id"],"left")

    #ratings generation
    sqlCtx.registerDataFrameAsTable(ratings, "ratings")
    temp_ratings = sqlCtx.sql("SELECT id \
    , CAST(1.0*star1/(star1+star2+star3+star4+star5) AS float) AS star1 \
    , CAST(1.0*star2/(star1+star2+star3+star4+star5) AS float) AS star2 \
    , CAST(1.0*star3/(star1+star2+star3+star4+star5) AS float) AS star3 \
    , CAST(1.0*star4/(star1+star2+star3+star4+star5) AS float) AS star4 \
    , CAST(1.0*star5/(star1+star2+star3+star4+star5) AS float) AS star5 \
    , (star1+star2+star3+star4+star5) AS num_ratings \
     FROM ratings")

    predictors = predictors.join(temp_ratings,["id"],"left")

    testing(predictors)
    
    # Categories
    list_categories = [ x.category.replace(" ","_") for x in sqlCtx.sql("SELECT category \
     FROM downloads\
     group by category \
     ").collect()]
    for cat in list_categories:
        sqlCtx.registerDataFrameAsTable(predictors, "predictors")
        predictors=sqlCtx.sql('''SELECT *, CASE WHEN (category = "'''+cat+'''") THEN 1 ELSE 0 END AS '''+cat+''' FROM predictors''')

    # Device
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "iphone") THEN 1 ELSE 0 END AS iphone FROM predictors''')
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    #predictors=sqlCtx.sql('''SELECT *, CASE WHEN (device = "ipad") THEN 1 ELSE 0 END AS ipad FROM predictors''')



    #Language of the title
    for l in lang:
        sqlCtx.registerDataFrameAsTable(predictors, "predictors")
        predictors=sqlCtx.sql('''SELECT *, CASE WHEN (get_language(name) = "'''+l+'''") THEN 1 \
        ELSE 0 END AS '''+l.replace("-","_")+''' FROM predictors''')


    #Reviews 
    #First step
    list_countries =['United_States', 'France', 'Japan', 'Spain', 'United_Kingdom','Saudi_Arabia', 'Germany'\
         , 'Hong_Kong', 'Switzerland', 'Turkey','Netherlands', 'Australia', 'Norway', 'Sweden', 'China', 'Canada'\
         ,'Tanzania', 'Denmark', 'South_Korea', 'Italy', 'Finland', 'Taiwan','Russia', 'Philippines', 'Slovenia'\
         , 'Ireland', 'Belgium', 'Mexico','Austria', 'India', 'Brazil', 'Benin', 'New_Zealand','United_Arab_Emirates'\
         , 'Ukraine', 'Poland', 'Israel', 'Portugal','Tunisia', 'Mali', 'Slovakia', 'Zimbabwe', 'Thailand', 'Panama'\
         ,'Indonesia', 'Singapore', 'Greece', 'Senegal', 'Nicaragua','Hungary', 'Czech_Republic', 'Macedonia', 'Chile'\
         , 'Uruguay','Malaysia', 'Algeria', 'Nepal', 'Mauritania', 'Croatia']

    cmd = '''review_rdd = reviews\
    .map(lambda x : (x.id , Row(id = x.id , avg_review = x.rating \
    , recent_review = get_recentReviews(x.date), nb_review = 1\
    ,version = set([x.version])'''
    cmd+=",country = set([x.country])"
    #for c in list_countries:
    #    cmd+=","+c+''' = int( escape(x.country) == "'''+c+'''")'''
    for i in range(1,6):
        cmd+=",review_rating_"+str(i)+" = int(x.rating == "+str(i)+")  "
    cmd+=")))"
    exec cmd in globals(), locals()

    #Group step
    cmd = '''review_rdd = review_rdd.reduceByKey(lambda x1 ,x2 : Row(\
     avg_review = x1.avg_review + x2.avg_review\
       ,recent_review = x1.recent_review + x2.recent_review, nb_review = x1.nb_review + x2.nb_review'''
    #for c in list_countries:
    #    cmd+=" , "+c+" = x1."+c+" + x2."+c
    cmd+=", country = x1.country.union(x2.country)"
    cmd+=", version = x1.version.union(x2.version)"
    for i in range(1,6):
        cmd+=", review_rating_"+str(i)+" = x1.review_rating_"+str(i)+" + x2.review_rating_"+str(i)+" "
    cmd+="))"
    exec cmd in globals(), locals()

    # Clean the grouped rdd
    cmd = '''review_rdd = review_rdd.map(lambda (id , x) : [ id \
    ,  1.0*x.avg_review /  x.nb_review\
       , x.recent_review,  x.nb_review'''
    #for c in list_countries:
    #    cmd+=" , 1.0* x."+c+"/ x.nb_review"
    cmd+=",  escape(x.country.pop())"
    cmd+=",  len(x.version) -1 " # -1 if want number of updates
    for i in range(1,6):
        cmd+=", 1.0*x.review_rating_"+str(i)+" / x.nb_review"
    cmd+="])"
    exec cmd in globals(), locals()

    #Put back into dataframe
    grp_reviews = sqlCtx.createDataFrame(review_rdd, ["id","avg_review"\
          ,"recent_reviews","nb_reviews","country","versions"]+["review_rating_"+str(i) for i in range(1,6)])
        #,"recent_reviews","nb_reviews"] + list_countries + ["versions"]+["review_rating_"+str(i) for i in range(1,6)])

    #Join with predictors 
    predictors = predictors.join(grp_reviews,["id"],"left").fillna("no_country",["country"])

    testing(predictors)

    # Generate DL Projection
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    derived_feats = sqlCtx.sql("SELECT id, device\
        ,(7*download_sum+cumulative_downloads_2015_02) AS dl_projection \
        ,CAST((1000000.0*num_ratings/(cumulative_downloads_2015_02 + download_sum))AS float )  AS ratings_per_downloads \
        ,CAST((1.0*num_ratings/(days_since_release+56))AS float )  AS ratings_per_day \
        ,CAST((1000000.0*nb_reviews/download_sum)AS float )  AS review_per_downloads \
        ,CAST((1.0*recent_reviews/nb_reviews)AS float )  AS review_recent_over_old \
        ,CAST((1.0*nb_reviews / days_since_release)AS float )  AS review_per_day \
        ,CAST((1.0*cumulative_downloads_2015_02/(days_since_release))AS float )  AS downloads_per_day_before \
        ,CAST(1.0*download_sum/cumulative_downloads_2015_02 AS float ) AS relative_download_increase\
        ,CAST(1.0*nb_reviews / (CASE WHEN (days_since_release<56) THEN days_since_release ELSE 56 END) AS float) AS reviews_per_days\
        ,CAST(7*download_sum+cumulative_downloads_2015_02 / days_since_release / days_since_release AS float) AS down_over_days2\
        FROM predictors")
    predictors = predictors.join(derived_feats,["id","device"],"left")


    #we could group by continent or use the market potential
    list_countries+=["no_country"]
    for co in list_countries:
        sqlCtx.registerDataFrameAsTable(predictors, "predictors")
        predictors=sqlCtx.sql('''SELECT *, CASE WHEN (country = "'''+co+'''") THEN 1 ELSE 0 END AS '''+co+''' FROM predictors''')


    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors=sqlCtx.sql('''SELECT *, get_market(country,device) AS market_size FROM predictors''')

    sqlCtx.registerDataFrameAsTable(rating_country, "rating_country")
    rating_country = sqlCtx.sql('SELECT id\
    , CAST(SUM(star1)*1.0/(SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5)) AS float) AS total_star1\
    , CAST(SUM(star2)*1.0/(SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5)) AS float)AS total_star2\
    , CAST(SUM(star3)*1.0/(SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5))AS float) AS total_star3\
    , CAST(SUM(star4)*1.0/(SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5))AS float) AS total_star4\
    , CAST(SUM(star5)*1.0/(SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5)) AS float)AS total_star5\
    , (SUM(star1)+SUM(star2)+SUM(star3)+SUM(star4)+SUM(star5)) AS total_star\
    , COUNT(1) AS nb_countries FROM rating_country GROUP BY id')

    sqlCtx.registerDataFrameAsTable(rating_country, "rating_country")

    predictors = predictors.join(rating_country,["id"],how='left_outer')

    testing(predictors)

    #More on the ratings
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    cmd = 'SELECT * '
    for i in range(1,6):
        cmd+=',(total_star'+str(i)+' * total_star / cumulative_downloads_2015_02) AS country_star'+str(i)+'_per_downloads '
        cmd+=',(review_rating_'+str(i)+' * nb_reviews / download_sum) AS review_star'+str(i)+'_per_downloads '
        cmd+=',(star'+str(i)+' * num_ratings / cumulative_downloads_2015_02) AS rating_star'+str(i)+'_per_downloads '
    cmd+= ' FROM predictors'

    predictors = sqlCtx.sql(cmd)


    sqlCtx.registerDataFrameAsTable(predictors, "predictors")
    predictors = sqlCtx.sql('SELECT *, CAST((max_step - min_step)*1.0/download_sum AS float) AS relative_step\
    , (max_step - min_step)  AS download_step \
    FROM predictors')

    testing(predictors)
    
    sqlCtx.registerDataFrameAsTable(predictors, "predictors")

    predictors = sqlCtx.sql('SELECT *, get_gdp(country)  AS gdp_country \
    FROM predictors')

    predictors = predictors.fillna(0)
    predictors.cache()

    testing(predictors)
    return predictors.toPandas().sort_values(by="id").fillna(0)




In [ ]:
predictors_train = generate_predictors(  'train_app_downloads.csv'\
                                      , 'train_app_rating.csv'\
                                      , 'train_usage.csv'\
                                      , 'train_revenue.csv'\
                                      , 'train_cumulative_downloads_2015-02.csv'\
                                      , 'train_release_date.csv'\
                                      , 'train_rating_by_country.csv'\
                                      , 'train_app_review.csv')
predictors_train.to_csv('/mnt/final_train_predictors.csv', encoding='utf8')

In [ ]:
predictors_test = generate_predictors(   'test_set/test_app_downloads.csv'\
                                      , 'test_set/test_app_rating.csv'\
                                      , 'test_set/test_usage.csv'\
                                      , 'test_set/test_revenue.csv'\
                                      , 'test_set/test_cumulative_downloads_2015-02.csv'\
                                      , 'test_set/test_release_date.csv'\
                                      , 'test_set/test_rating_by_country.csv'\
                                      , 'test_set/test_app_review.csv')
predictors_test.to_csv('/mnt/final_test_predictors.csv', encoding='utf8')

## To Pandas


In [1]:
try:
    predictors_train = pd.read_csv('/mnt/final_train_predictors.csv').drop('Unnamed: 0',1)
    predictors_test = pd.read_csv('/mnt/final_test_predictors.csv').drop('Unnamed: 0',1)
except:
    print "Cannot load files on s3"

Cannot load files on s3


In [2]:
predictors_train = predictors_train.sort_values(["id","device"])
predictors_test = predictors_test.sort_values(["id","device"])
output_train = pd.merge(predictors_train,output.toPandas(),on = ["id","device"],how="left")\
[["id","device", "cumulative_downloads_2016_02"]]

numerical_predictors = list(predictors_train.columns.values)
#numerical_predictors.remove("id");
numerical_predictors.remove("device");
numerical_predictors.remove("category");
numerical_predictors.remove("name");
numerical_predictors.remove("country");


predictors_train_id = predictors_train[["id","device"]].as_matrix()
predictors_test_id = predictors_test[["id","device"]].as_matrix()



#This is the metric we use to determine our performance
def metric(y_pred,y_test,percent=1.0):
    top = int(len(y_pred)/100.0*percent)
    return (len(set([i[0] for i in sorted(enumerate(y_pred), key=lambda x:x[1],reverse=True)][0:top])
       .intersection([i[0] for i in sorted(enumerate(y_test), key=lambda x:x[1],reverse=True)][0:top])
               ))*(100.0)/top

NameError: name 'predictors_train' is not defined

In [ ]:
def final_model(predictors_train,output_train,predictors_test):
    print '.',
    with_rel = False
    top_percent_classif = 12
    split_id = 771335033
    
    train_features_1 = predictors_train[numerical_predictors_1].as_matrix()
    train_features_2 = predictors_train[predictors_train.id >= split_id][numerical_predictors_2].as_matrix()
    train_output_1 = output_train["cumulative_downloads_2016_02"].as_matrix()
    train_output_2 = output_train[output_train.id >= split_id]["cumulative_downloads_2016_02"].as_matrix()



    test_features_1 = predictors_test[predictors_test.id < split_id][numerical_predictors_1].as_matrix()
    test_features_2 = predictors_test[predictors_test.id >= split_id][numerical_predictors_2].as_matrix()
    
     ###Classification
    #mod_class_1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    #mod_class_1.fit(train_features_1, train_output_1)
    #y_pred_1 =  mod_class_1.predict(test_features_1)
    #mask_1 = y_pred_1 > (sorted(y_pred_1,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_1))]

    #mod_class_2= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    #mod_class_2.fit(train_features_2, train_output_2)
    #y_pred_2 =  mod_class_2.predict(test_features_2)
    #mask_2 = y_pred_2 > (sorted(y_pred_2,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_2))]

    mod_class= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    mod_class.fit(np.concatenate((train_features_1,train_features_2)), np.concatenate((train_output_1,train_output_2)))
    y_pred_1 =  mod_class.predict(test_features_1)
    mask_1 = y_pred_1 > (sorted(y_pred_1,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_1))]

    y_pred_2 =  mod_class.predict(test_features_2)
    mask_2 = y_pred_2 > (sorted(y_pred_2,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_2))]
    print '.',

    ###Weights
    threshold1_20 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*20.0/100)]
    threshold1_12 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*12.0/100)]
    threshold1_5 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*5.0/100)]
    threshold1_1 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*1.0/100)]
    threshold1_0 = 0.0
    if with_rel == True :
        relevance_1 =(range(len(train_features_1)))
        relevance_1.sort(key=lambda x: (train_output_1[x]))
        relevance_1 = (np.asarray(relevance_1)-int(99.0*len(train_features_1)/100))
        relevance_1[relevance_1<0] = 0

        mod_train_1 = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,max_depth=2, random_state=0, loss='ls')\
        .fit(train_features_1,np.log(train_output_1))
        train_pred_1 =  mod_train_1.predict(train_features_1)

        pred_train_mask_1 = train_pred_1 > sorted(train_pred_1,reverse = True)[int(1.0/100*len(train_features_1))]
        true_train_mask_1 = train_output_1 > sorted(train_output_1,reverse = True)[int(1.0/100*len(train_features_1))]
        hard_class_1 = np.logical_xor(pred_train_mask_1,true_train_mask_1)
    else: 
        relevance_1 =(np.zeros(len(train_features_1)))
        hard_class_1 =(np.zeros(len(train_features_1)))

    print '.',
    weights_1\
    = 0.05*(train_output_1 >  threshold1_0).astype(int)\
    + 0.05*(train_output_1 >  threshold1_20).astype(int)\
    + 0.5*(train_output_1 >  threshold1_12).astype(int)\
    + 0*(train_output_1 >  threshold1_5).astype(int)\
    + 0*(train_output_1 >  threshold1_1).astype(int)\
    + 0*relevance_1\
    + 0*hard_class_1\
    + 0*(np.asarray([0]*(len(train_features_1) -len(train_features_2)) + [1]*len(train_features_2) ))\
       * (train_output_1 >  threshold1_12).astype(int)
    + 1.5*(np.asarray([0]*(len(train_features_1) -len(train_features_2)) + [1]*len(train_features_2) ))


    mod_1 = GradientBoostingRegressor(max_features =1.0,n_estimators=100, learning_rate=0.1,max_depth=2, random_state=0, loss='ls')
    #mod_1 = linear_model.Lasso(alpha=100)
    mod_1.fit(train_features_1, train_output_1 \
              ,sample_weight \
    =(weights_1)*1.0/sum(weights_1))
    print '.',


    threshold2_20 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*20.0/100)]
    threshold2_12 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*12.0/100)]
    threshold2_5 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*5.0/100)]
    threshold2_1 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*1.0/100)]
    threshold2_0 = 0.0

    if with_rel == True :
        relevance_2 =(range(len(train_features_2)))
        relevance_2.sort(key=lambda x: (train_output_2[x]))
        relevance_2 = (np.asarray(relevance_2)-int(99.0*len(train_features_2)/100))
        relevance_2[relevance_2<0] = 0

        mod_train_2 = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1,max_depth=2, random_state=0, loss='ls')\
        .fit(train_features_2,np.log(train_output_2))
        train_pred_2 =  mod_train_2.predict(train_features_2)

        pred_train_mask_2 = train_pred_2 > sorted(train_pred_2,reverse = True)[int(1.0/100*len(train_features_2))]
        true_train_mask_2 = train_output_2 > sorted(train_output_2,reverse = True)[int(1.0/100*len(train_features_2))]
        hard_class_2 = np.logical_xor(pred_train_mask_2,true_train_mask_2)
    else: 
        relevance_2 =(np.zeros(len(train_features_2)))
        hard_class_2 =(np.zeros(len(train_features_2)))
    print '.',

    weights_2 \
    = 1*(train_output_2 >  threshold2_0).astype(int)\
    + 0*(train_output_2 >  threshold2_20).astype(int)\
    + 0.5*(train_output_2 >  threshold2_12).astype(int)\
    + 0*(train_output_2 >  threshold2_5).astype(int)\
    + 0*(train_output_2 >  threshold2_1).astype(int)\
    + 0*relevance_2*relevance_2\
    + 0*hard_class_2




    mod_2 = GradientBoostingRegressor(n_estimators=500, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')
    #mod_2 = linear_model.Lasso(alpha=100)
    #mod_2.fit(train_features_2, train_output_2)
    mod_2.fit(train_features_2, train_output_2 ,sample_weight =(weights_2)*1.0/sum(weights_2))
    print '.',

    y_pred_1 = mod_1.predict(test_features_1)
    print '.',

    y_pred_2 = mod_2.predict(test_features_2)

    ###CHECK ON THE IDs
    #print min(train_features_2[:,0])
    #print min(test_features_2[:,0])

    ###Taking only the top 1 and not the values (not working)
    #y_pred_1 = (y_pred_1 > sorted(y_pred_1,reverse = True)[int(len(y_pred_1)*0.6/100)])
    #y_pred_2 = (y_pred_2 > sorted(y_pred_2,reverse = True)[int(len(y_pred_2)*2.4/100)])

    #print "lenghts of train and test : " + str(len(train_features_1) ) + " ; "+ str(len(test_output_1))
    #print "Fraction in test_2 : "+str(1.0*sum(test_output_2 > test_threshold)/sum(np.concatenate((test_output_1,test_output_2)) > test_threshold))

    print '.',
    mod = GradientBoostingRegressor(max_features =1.0,n_estimators=100, learning_rate=0.15,max_depth=3, random_state=0, loss='ls')
    mod.fit(train_features_1, train_output_1)
    pred_boost = mod.predict(np.concatenate((test_features_1,test_features_2)))
    print '.',



    #print "perf_1 = "+str(metric(test_output_1,y_pred_1*mask_1,0.6))
    #print "perf_2 = "+str(metric(test_output_2,y_pred_2*mask_2,2.4))
    y_pred = np.concatenate((y_pred_1*mask_1,y_pred_2*mask_2))+pred_boost


    return y_pred 

In [ ]:
# The predictor pandas needs to be sorted by ascending ID

### Obtained with boosting feature selection
#numerical_predictors_2 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','week_7','downloads_std','coef_0','versions','download_sum','week_4','relative_download_increase','total_star3','days_since_release','market_size','week_3','review_rating_5','star3','review_rating_4','week_1','coef_1','m4_mean','review_rating_2','week_2','Uruguay','avg_review','nb_reviews']
#numerical_predictors_1 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','coef_3','versions','downloads_std','download_sum','days_since_release','m2_mean','coef_1','m1_mean','week_7','m2_coef_1','review_rating_5','week_6','review_recent_over_old','week_2','star4','coef_0','total_star3','m1_max','m3_max','star3','review_per_downloads','week_4','market_size','review_rating_4','week_3','nb_reviews','nb_missing','daily_avg','ratings_per_day']

###Custom
#numerical_predictors_2 = ['download_sum', 'nb_reviews', 'iphone', 'Education']
#numerical_predictors_1 = ['dl_projection', 'm3_max', 'week_8', 'coef_2', 'review_star5_per_downloads']

### Use all predictors
numerical_predictors_2 = list(numerical_predictors)
numerical_predictors_1 = list(numerical_predictors)

#numerical_predictors_1 = ["coef_0","coef_1","coef_2"]
#numerical_predictors_2 = ["coef_0","coef_1","coef_2"]

#numerical_predictors_1 = lst
#numerical_predictors_2 = lst

#Analysis on train performance if train > test this means there is overfit


K = 5
for mu in range(200000):
    #np.random.seed(mu)

    kf = KFold(len(predictors_train), n_folds=K,shuffle=True)
    new_top = []
    err_1 = []
    err_2 = []
    
    for train, test in kf:
        

        ### The model_1 is trainned on everything


        #test_output_1 = output_train.iloc[test][output_train.iloc[test].id < split_id]["cumulative_downloads_2016_02"].as_matrix()
        #test_output_2 = output_train.iloc[test][output_train.iloc[test].id >= split_id]["cumulative_downloads_2016_02"].as_matrix()
        test_output = output_train.iloc[test]["cumulative_downloads_2016_02"].as_matrix()
        
        #test_threshold = sorted(np.concatenate((test_output_1,test_output_2)),reverse = True)[int(len(test)*1.0/100)]
        

        ########
        y_pred = final_model(predictors_train.iloc[train],output_train.iloc[train],predictors_train.iloc[test])
        ########

        #print "perf_1 = "+str(metric(test_output_1,y_pred_1,0.6))
        #print "perf_2 = "+str(metric(test_output_2,y_pred_2,2.4))
        #y_pred = np.concatenate((y_pred_1,y_pred_2))

        #err_1.append(metric(test_output_1,y_pred_1,0.6))
        #err_2.append(metric(test_output_2,y_pred_2,2.4))
        err = (metric(test_output,y_pred))
        new_top.append(err)
        print "perf concat : "+str(err)
        
        
        
        

    print "  =========== "+str(mu)
    #print "Total CV_1 : "+str(1.0*sum(err_1)/len(err_1))
    #print "Total CV_2 : "+str(1.0*sum(err_2)/len(err_2))
    print "Total CV      ---------+>   : "+str(1.0*sum(new_top)/len(new_top))
    print " --- "



In [ ]:
# The predictor pandas needs to be sorted by ascending ID

### Obtained with boosting feature selection
#numerical_predictors_2 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','week_7','downloads_std','coef_0','versions','download_sum','week_4','relative_download_increase','total_star3','days_since_release','market_size','week_3','review_rating_5','star3','review_rating_4','week_1','coef_1','m4_mean','review_rating_2','week_2','Uruguay','avg_review','nb_reviews']
#numerical_predictors_1 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','coef_3','versions','downloads_std','download_sum','days_since_release','m2_mean','coef_1','m1_mean','week_7','m2_coef_1','review_rating_5','week_6','review_recent_over_old','week_2','star4','coef_0','total_star3','m1_max','m3_max','star3','review_per_downloads','week_4','market_size','review_rating_4','week_3','nb_reviews','nb_missing','daily_avg','ratings_per_day']

###Custom
#numerical_predictors_2 = ['download_sum', 'nb_reviews', 'iphone', 'Education']
#numerical_predictors_1 = ['dl_projection', 'm3_max', 'week_8', 'coef_2', 'review_star5_per_downloads']

### Use all predictors
numerical_predictors_2 = list(numerical_predictors)
numerical_predictors_1 = list(numerical_predictors)

#numerical_predictors_1 = ["coef_0","coef_1","coef_2"]
#numerical_predictors_2 = ["coef_0","coef_1","coef_2"]

#Analysis on train performance if train > test this means there is overfit

np.random.seed(1600)


########
y_pred = final_model(predictors_train,output_train,predictors_test)
########

print "   "
print "   "
print "   "


thres = sorted(y_pred,reverse=True)[int(len(y_pred)*1.0/100)]
lst= []
name_lst = []
for x in enumerate(y_pred):
    if x[1]>thres:
        lst.append( predictors_test_id[x[0]])
        name_lst.append((x[1],predictors_test[predictors_test.id == predictors_test_id[x[0]][0]]\
                     [predictors_test[predictors_test.id == predictors_test_id[x[0]][0]]\
                      .device == predictors_test_id[x[0]][1]]['name'].iloc[0]))

for x in lst:
    print str(x[0]) + ", "+x[1]

print "   "
print "   "
print "   "

for x in sorted(name_lst,reverse=True):
    print str(x[0]) + " \t ---> "+x[1]


In [ ]:

thres = sorted(y_pred,reverse=True)[int(len(y_pred)*1.0/100)]
lst= []
name_lst = []
for x in enumerate(y_pred):
    if x[1]>thres:
        lst.append( predictors_test_id[x[0]])
        name_lst.append((x[1],predictors_test[predictors_test.id == predictors_test_id[x[0]][0]]\
                     [predictors_test[predictors_test.id == predictors_test_id[x[0]][0]]\
                      .device == predictors_test_id[x[0]][1]]['name'].iloc[0]))


In [ ]:
for x in lst:
    print str(x[0]) + ", "+x[1]

print "   "
print "   "
print "   "

for x in sorted(name_lst,reverse=True):
    print str(x[0]) + " \t ---> "+x[1]

In [ ]:
def final_model(train_features_1,train_features_2,test_features_1,test_features_2,train_output_1,train_output_2):
    ###Classification
    #mod_class_1= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    #mod_class_1.fit(train_features_1, train_output_1)
    #y_pred_1 =  mod_class_1.predict(test_features_1)
    #mask_1 = y_pred_1 > (sorted(y_pred_1,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_1))]

    #mod_class_2= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    #mod_class_2.fit(train_features_2, train_output_2)
    #y_pred_2 =  mod_class_2.predict(test_features_2)
    #mask_2 = y_pred_2 > (sorted(y_pred_2,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_2))]

    mod_class= RandomForestRegressor(max_features = 'sqrt',n_estimators = 100)   
    mod_class.fit(np.concatenate((train_features_1,train_features_2)), np.concatenate((train_output_1,train_output_2)))
    y_pred_1 =  mod_class.predict(test_features_1)
    mask_1 = y_pred_1 > (sorted(y_pred_1,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_1))]

    y_pred_2 =  mod_class.predict(test_features_2)
    mask_2 = y_pred_2 > (sorted(y_pred_2,reverse = True))[int(top_percent_classif*1.0/100*len(y_pred_2))]


    ###Weights
    threshold1_20 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*20.0/100)]
    threshold1_12 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*12.0/100)]
    threshold1_5 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*5.0/100)]
    threshold1_1 = sorted(train_output_1,reverse = True)[int(len(train_output_1)*1.0/100)]
    threshold1_0 = 0.0
    if with_rel == True :
        relevance_1 =(range(len(train_features_1)))
        relevance_1.sort(key=lambda x: (train_output_1[x]))
        relevance_1 = (np.asarray(relevance_1)-int(99.0*len(train_features_1)/100))
        relevance_1[relevance_1<0] = 0

        mod_train_1 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
        .fit(train_features_1,np.log(train_output_1))
        train_pred_1 =  mod_train_1.predict(train_features_1)

        pred_train_mask_1 = train_pred_1 > sorted(train_pred_1,reverse = True)[int(1.0/100*len(train_features_1))]
        true_train_mask_1 = train_output_1 > sorted(train_output_1,reverse = True)[int(1.0/100*len(train_features_1))]
        hard_class_1 = np.logical_xor(pred_train_mask_1,true_train_mask_1)
    else: 
        relevance_1 =(np.zeros(len(train_features_1)))
        hard_class_1 =(np.zeros(len(train_features_1)))


    weights_1\
    = 0*(train_output_1 >  threshold1_0).astype(int)\
    + 0*(train_output_1 >  threshold1_20).astype(int)\
    + 0.5*(train_output_1 >  threshold1_12).astype(int)\
    + 0*(train_output_1 >  threshold1_5).astype(int)\
    + 0*(train_output_1 >  threshold1_1).astype(int)\
    + 0*relevance_1\
    + 0*hard_class_1\
    + 1.5*np.asarray([0]*(len(train_features_1) -len(train_features_2)) + [1]*len(train_features_2) )#* (train_output_1 >  threshold1_12).astype(int)




    mod_1 = GradientBoostingRegressor(max_features =1.0,n_estimators=100, learning_rate=0.1,max_depth=2, random_state=0, loss='ls')
    #mod_1 = linear_model.Lasso(alpha=100)
    mod_1.fit(train_features_1, train_output_1 \
              ,sample_weight \
    =(weights_1)*1.0/sum(weights_1))

    y_pred_1 = mod_1.predict(test_features_1)

    threshold2_20 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*20.0/100)]
    threshold2_12 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*12.0/100)]
    threshold2_5 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*5.0/100)]
    threshold2_1 = sorted(train_output_2,reverse = True)[int(len(train_output_2)*1.0/100)]
    threshold2_0 = 0.0

    if with_rel == True :
        relevance_2 =(range(len(train_features_2)))
        relevance_2.sort(key=lambda x: (train_output_2[x]))
        relevance_2 = (np.asarray(relevance_2)-int(99.0*len(train_features_2)/100))
        relevance_2[relevance_2<0] = 0

        mod_train_2 = GradientBoostingRegressor(n_estimators=500, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
        .fit(train_features_2,np.log(train_output_2))
        train_pred_2 =  mod_train_2.predict(train_features_2)

        pred_train_mask_2 = train_pred_2 > sorted(train_pred_2,reverse = True)[int(1.0/100*len(train_features_2))]
        true_train_mask_2 = train_output_2 > sorted(train_output_2,reverse = True)[int(1.0/100*len(train_features_2))]
        hard_class_2 = np.logical_xor(pred_train_mask_2,true_train_mask_2)
    else: 
        relevance_2 =(np.zeros(len(train_features_2)))
        hard_class_2 =(np.zeros(len(train_features_2)))


    weights_2 \
    = 1*(train_output_2 >  threshold2_0).astype(int)\
    + 0*(train_output_2 >  threshold2_20).astype(int)\
    + 0*(train_output_2 >  threshold2_12).astype(int)\
    + 0*(train_output_2 >  threshold2_5).astype(int)\
    + 0*(train_output_2 >  threshold2_1).astype(int)\
    + 0*relevance_2*relevance_2\
    + 0*hard_class_2




    mod_2 = GradientBoostingRegressor(n_estimators=50, learning_rate=0.1,max_depth=3, random_state=0, loss='ls')
    #mod_2 = linear_model.Lasso(alpha=100)
    #mod_2.fit(train_features_2, train_output_2)
    mod_2.fit(train_features_2, train_output_2 ,sample_weight =(weights_2)*1.0/sum(weights_2))

    y_pred_2 = mod_2.predict(test_features_2)

    ###CHECK ON THE IDs
    #print min(train_features_2[:,0])
    #print min(test_features_2[:,0])

    ###Taking only the top 1 and not the values (not working)
    #y_pred_1 = (y_pred_1 > sorted(y_pred_1,reverse = True)[int(len(y_pred_1)*0.6/100)])
    #y_pred_2 = (y_pred_2 > sorted(y_pred_2,reverse = True)[int(len(y_pred_2)*2.4/100)])

    #print "lenghts of train and test : " + str(len(train_features_1) ) + " ; "+ str(len(test_output_1))
    #print "Fraction in test_2 : "+str(1.0*sum(test_output_2 > test_threshold)/sum(np.concatenate((test_output_1,test_output_2)) > test_threshold))


    return y_pred_1*mask_1, y_pred_2*mask_2

In [ ]:
# The predictor pandas needs to be sorted by ascending ID

### Obtained with boosting feature selection
#numerical_predictors_2 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','week_7','downloads_std','coef_0','versions','download_sum','week_4','relative_download_increase','total_star3','days_since_release','market_size','week_3','review_rating_5','star3','review_rating_4','week_1','coef_1','m4_mean','review_rating_2','week_2','Uruguay','avg_review','nb_reviews']
#numerical_predictors_1 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','coef_3','versions','downloads_std','download_sum','days_since_release','m2_mean','coef_1','m1_mean','week_7','m2_coef_1','review_rating_5','week_6','review_recent_over_old','week_2','star4','coef_0','total_star3','m1_max','m3_max','star3','review_per_downloads','week_4','market_size','review_rating_4','week_3','nb_reviews','nb_missing','daily_avg','ratings_per_day']

###Custom
#numerical_predictors_2 = ['download_sum', 'nb_reviews', 'iphone', 'Education']
#numerical_predictors_1 = ['dl_projection', 'm3_max', 'week_8', 'coef_2', 'review_star5_per_downloads']

### Use all predictors
numerical_predictors_2 = list(numerical_predictors)
numerical_predictors_1 = list(numerical_predictors)

#numerical_predictors_1 = ["coef_0","coef_1","coef_2"]
#numerical_predictors_2 = ["coef_0","coef_1","coef_2"]

#Analysis on train performance if train > test this means there is overfit

with_rel = False
K = 5
for mu in ['nope']:
    np.random.seed(1)
    top_percent_classif = 12
    split_id = 771335033
    kf = KFold(len(predictors_train), n_folds=K,shuffle=True)
    new_top = []
    err_1 = []
    err_2 = []
    for train, test in kf:
        

        ### The model_1 is trainned on everything

        train_features_1 = predictors_train.iloc[train][numerical_predictors_1].as_matrix()
        train_features_2 = predictors_train.iloc[train][predictors_train.iloc[train].id >= split_id][numerical_predictors_2].as_matrix()
        train_output_1 = output_train.iloc[train]["cumulative_downloads_2016_02"].as_matrix()
        train_output_2 = output_train.iloc[train][output_train.iloc[train].id >= split_id]["cumulative_downloads_2016_02"].as_matrix()
        


        test_features_1 = predictors_train.iloc[test][predictors_train.iloc[test].id < split_id][numerical_predictors_1].as_matrix()
        test_features_2 = predictors_train.iloc[test][predictors_train.iloc[test].id >= split_id][numerical_predictors_2].as_matrix()
        test_output_1 = output_train.iloc[test][output_train.iloc[test].id < split_id]["cumulative_downloads_2016_02"].as_matrix()
        test_output_2 = output_train.iloc[test][output_train.iloc[test].id >= split_id]["cumulative_downloads_2016_02"].as_matrix()
        
        test_threshold = sorted(np.concatenate((test_output_1,test_output_2)),reverse = True)[int(len(test)*1.0/100)]
        

        ########
        y_pred_1 , y_pred_2 = final_model(train_features_1,train_features_2,test_features_1,test_features_2,train_output_1,train_output_2)
        ########

        print "perf_1 = "+str(metric(test_output_1,y_pred_1,0.6))
        print "perf_2 = "+str(metric(test_output_2,y_pred_2,2.4))
        y_pred = np.concatenate((y_pred_1,y_pred_2))

        err_1.append(metric(test_output_1,y_pred_1,0.6))
        err_2.append(metric(test_output_2,y_pred_2,2.4))
        err = (metric(np.concatenate((test_output_1,test_output_2)),y_pred))
        new_top.append(err)
        print "perf concat : "+str(err)
        
        
        
        

    print "  =========== "+str(mu)
    print "Total CV_1 : "+str(1.0*sum(err_1)/len(err_1))
    print "Total CV_2 : "+str(1.0*sum(err_2)/len(err_2))
    print "Total CV : "+str(1.0*sum(new_top)/len(new_top))
    print " --- "



In [ ]:
warnings.filterwarnings('error')

In [ ]:
# The predictor pandas needs to be sorted by ascending ID

### Obtained with boosting feature selection
#numerical_predictors_2 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','week_7','downloads_std','coef_0','versions','download_sum','week_4','relative_download_increase','total_star3','days_since_release','market_size','week_3','review_rating_5','star3','review_rating_4','week_1','coef_1','m4_mean','review_rating_2','week_2','Uruguay','avg_review','nb_reviews']
#numerical_predictors_1 = ['dl_projection','week_8','coef_2','week_5','downloads_per_day_before','cumulative_downloads_2015_02','coef_3','versions','downloads_std','download_sum','days_since_release','m2_mean','coef_1','m1_mean','week_7','m2_coef_1','review_rating_5','week_6','review_recent_over_old','week_2','star4','coef_0','total_star3','m1_max','m3_max','star3','review_per_downloads','week_4','market_size','review_rating_4','week_3','nb_reviews','nb_missing','daily_avg','ratings_per_day']

###Custom
#numerical_predictors_2 = ['download_sum', 'nb_reviews', 'iphone', 'Education']
#numerical_predictors_1 = ['dl_projection', 'm3_max', 'week_8', 'coef_2', 'review_star5_per_downloads']

### Use all predictors
numerical_predictors_2 = list(numerical_predictors)
numerical_predictors_1 = list(numerical_predictors)

#numerical_predictors_1 = ["coef_0","coef_1","coef_2"]
#numerical_predictors_2 = ["coef_0","coef_1","coef_2"]

#Analysis on train performance if train > test this means there is overfit

with_rel = False

np.random.seed(0)
top_percent_classif = 12
split_id = 771335033

train_features_1 = predictors_train[predictors_train.id < split_id][numerical_predictors_1].as_matrix()
train_features_2 = predictors_train[predictors_train.id >= split_id][numerical_predictors_2].as_matrix()
train_output_1 = output_train[output_train.id < split_id]["cumulative_downloads_2016_02"].as_matrix()
train_output_2 = output_train[output_train.id >= split_id]["cumulative_downloads_2016_02"].as_matrix()



test_features_1 = predictors_test[predictors_test.id < split_id][numerical_predictors_1].as_matrix()
test_features_2 = predictors_test[predictors_test.id >= split_id][numerical_predictors_2].as_matrix()

########
y_pred_1 , y_pred_2 = final_model(train_features_1,train_features_2,test_features_1,test_features_2,train_output_1,train_output_2)
########

y_pred = np.concatenate((y_pred_1,y_pred_2))

thres = sorted(y_pred,reverse=True)[int(len(y_pred)*1.0/100)]
lst= []
for x in enumerate(y_pred):
    if x[1]>thres:
        lst.append( predictors_test_id[x[0]])

for x in lst:
    print str(x[0]) + ", "+x[1]





In [ ]:
for x in lst:
    print predictors_test[predictors_test.id == x[0] ]['name'].iloc[0]

In [ ]:
down = 'test_set/test_app_downloads.csv'
rati = 'test_set/test_app_rating.csv'
usag='test_set/test_usage.csv'
reve = 'test_set/test_revenue.csv'
prev ='test_set/test_cumulative_downloads_2015-02.csv'
rele = 'test_set/test_release_date.csv'
raco = 'test_set/test_rating_by_country.csv'
revi ='test_set/test_app_review.csv'


'''down = 'train_app_downloads.csv'
rati = 'train_app_rating.csv'
usag='train_usage.csv'
reve = 'train_revenue.csv'
prev ='train_cumulative_downloads_2015-02.csv'
rele = 'train_release_date.csv'
raco = 'train_rating_by_country.csv'
revi ='train_app_review.csv'
'''

sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])

usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/'+usag).drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/'+reve).drop('')

###imputations
#usage imputation
imputer = Imputer(missing_values=-1, strategy='median', axis=0)
pd_usages = usages.toPandas()
category = list(set(pd_usages["category"].values))
imp = pd.DataFrame(columns = pd_usages.columns)
for cat in category:
    #for dev in ["iphone","ipad"]:
        for metric in range(1,5):
            curr_df = pd_usages.ix[pd_usages["category"]==cat,:]
            #curr_df = curr_df.ix[curr_df["device"]==dev,:]
            curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            
            try:
                name = curr_df.columns
                df1 = curr_df.ix[:,0:6]
                df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
                df2.index = df1.index
                curr_df = pd.concat([df1,df2],axis = 1)
                curr_df.columns = name 
            except:
                pass
            
            
            imp = pd.concat([imp,curr_df],axis = 0)
usages = sqlCtx.createDataFrame(imp)

#revenue imputation
pd_revenues = revenues.toPandas()
imp = pd.DataFrame(columns = revenues.columns)
for cat in category:
    for dev in ["iphone","ipad"]:
        #for metric in range(1,5):
            curr_df = pd_revenues.ix[pd_revenues["category"]==cat,:]
            curr_df = curr_df.ix[curr_df["device"]==dev,:]
            #curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            name = curr_df.columns
            df1 = curr_df.ix[:,0:5]
            df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,5:]))
            df2.index = df1.index
            curr_df = pd.concat([df1,df2],axis = 1)
            curr_df.columns = name 
            imp = pd.concat([imp,curr_df],axis = 0)
revenues = sqlCtx.createDataFrame(imp)


temp_revenues = sqlCtx.sql("SELECT id, name, category, device, \
    get_revenue_coefficients(0,"+",".join(revenues.columns[4:])+") AS rev_coef_0, \
    get_revenue_coefficients(1,"+",".join(revenues.columns[4:])+") AS rev_coef_1, \
    get_revenue_coefficients(2,"+",".join(revenues.columns[4:])+") AS rev_coef_2, \
    get_revenue_max(0,"+",".join(revenues.columns[4:])+") AS rev_max, \
    get_revenue_mean(0,"+",".join(revenues.columns[4:])+") AS rev_mean FROM revenues")

temp_revenues.toPandas().head(1)
print ",".join(revenues.columns[4:])

In [ ]:
sqlCtx.registerDataFrameAsTable(usages, "usages")
sqlCtx.sql('select * from usages where id =411796932').fillna(-1).head()

In [ ]:
usag='test_set/test_usage.csv'
#usag='train_usage.csv'

usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/'+usag).drop('')



    
###imputations
#usage imputation
imputer = Imputer(missing_values=-1, strategy='median', axis=0)
pd_usages = usages.toPandas()
category = list(set(pd_usages["category"].values))
imp = pd.DataFrame(columns = pd_usages.columns)
for cat in category:
    #for dev in ["iphone","ipad"]:
        for metric in range(1,5):
            curr_df = pd_usages.ix[pd_usages["category"]==cat,:]
            #curr_df = curr_df.ix[curr_df["device"]==dev,:]
            curr_df = curr_df.ix[curr_df["metric"]==metric,:]
            try:
                name = curr_df.columns
                df1 = curr_df.ix[:,0:6]
                df2 = pd.DataFrame(imputer.fit_transform(curr_df.ix[:,6:]))
                df2.index = df1.index
                curr_df = pd.concat([df1,df2],axis = 1)
                curr_df.columns = name 
            except:
                pass
            imp = pd.concat([imp,curr_df],axis = 0)
usages = sqlCtx.createDataFrame(imp).fillna(-1)

m_col = '03_01_2015,03_08_2015,03_15_2015,03_22_2015,03_29_2015,04_05_2015,04_12_2015,04_19_2015'

m1 = sqlCtx.sql("SELECT * FROM usages WHERE metric = 1 AND id != 411796932")
sqlCtx.registerDataFrameAsTable(m1,"m1")
temp_m1 = sqlCtx.sql("SELECT id, name, category, device, \
get_usage_coefficients(0,"+m_col+") AS m1_coef_0, \
get_usage_coefficients(1,"+m_col+") AS m1_coef_1, \
get_usage_coefficients(2,"+m_col+") AS m1_coef_2, \
get_usage_max(0,"+m_col+") AS m1_max, \
get_usage_mean(0,"+m_col+") AS m1_mean FROM m1")

temp_m1.toPandas().head(1)
print "At last"

In [ ]:
",".join(m1.columns)

In [ ]:
down = 'test_set/test_app_downloads.csv'
rati = 'test_set/test_app_rating.csv'
usag='test_set/test_usage.csv'
reve = 'test_set/test_revenue.csv'
prev ='test_set/test_cumulative_downloads_2015-02.csv'
rele = 'test_set/test_release_date.csv'
raco = 'test_set/test_rating_by_country.csv'
revi ='test_set/test_app_review.csv'


'''down = 'train_app_downloads.csv'
rati = 'train_app_rating.csv'
usag='train_usage.csv'
reve = 'train_revenue.csv'
prev ='train_cumulative_downloads_2015-02.csv'
rele = 'train_release_date.csv'
raco = 'train_rating_by_country.csv'
revi ='train_app_review.csv'
'''

sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", os.environ['AWS_ACCESS_KEY_ID'])
sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", os.environ['AWS_SECRET_ACCESS_KEY'])

usages = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/'+usag).drop('')
revenues = sqlCtx.read \
    .format('com.databricks.spark.csv') \
    .options(header='true',inferSchema='true') \
    .load('s3n://cs341bucket1/Data/'+reve).drop('')

sqlCtx.registerDataFrameAsTable(revenues, "revenues")

In [ ]:
predictors_train

In [ ]:
predictors_train.head()

In [ ]:
mod_0 = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01,max_depth=2, random_state=0, loss='ls')\
        .fit(predictors_train[numerical_predictors].as_matrix(),output_train["cumulative_downloads_2016_02"].as_matrix())
ranking = mod_0.feature_importances_
lst = []
for e in [(i[0],i[1]) for i in sorted(zip(numerical_predictors,ranking),key=lambda x: -x[1])]:
    print e
    if e[1]>0:
        lst.append(e[0])
print lst

In [ ]:
warnings.filterwarnings('error')